In [ ]:
import SimpleITK as sitk
import numpy as np

# Load the segmentation image
segmentation_path = '/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr/IF_Tr_0004.nii.gz'
segmentation_image = sitk.ReadImage(segmentation_path)

In [ ]:
# --- Myocardium Thickness Calculation ---

# The label value for the myocardium in your segmentation
myocardium_label = 1

# Create a binary image for the myocardium
myocardium_mask = sitk.BinaryThreshold(segmentation_image,
                                      lowerThreshold=myocardium_label,
                                      upperThreshold=myocardium_label,
                                      insideValue=1,
                                      outsideValue=0)

# Calculate the signed Maurer distance map.
# This computes the Euclidean distance to the closest boundary for each voxel.
distance_map = sitk.SignedMaurerDistanceMap(myocardium_mask,
                                            insideIsPositive=True,
                                            squaredDistance=False,
                                            useImageSpacing=True)

# The thickness can be approximated by finding the maximum distance
# within the segmented region, which corresponds to half the thickness.
# We can then multiply by 2 to get the full thickness.
# A more robust approach is to analyze the distribution of distances.

# For a simple average thickness:
# Get the numpy array from the distance map
distance_array = sitk.GetArrayViewFromImage(distance_map)
myocardium_mask_array = sitk.GetArrayViewFromImage(myocardium_mask)

# Get the distance values only within the myocardium
myocardium_distances = distance_array[myocardium_mask_array == 1]

# The average "radius" is the mean of these distances
average_radius = np.mean(myocardium_distances)
average_thickness = average_radius * 2

print(f"Average Myocardium Thickness: {average_thickness} mm")

# Maximum thickness
max_radius = np.max(myocardium_distances)
max_thickness = max_radius * 2
print(f"Maximum Myocardium Thickness: {max_thickness} mm")

In [ ]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import nibabel as nib # Use nibabel to load your NIfTI files

def calculate_and_visualize_thickness(segmentation_filepath, label_of_interest):
    """
    Calculates and visualizes the thickness of a segmented object in a NIfTI file.

    Args:
        segmentation_filepath (str): Path to the NIfTI segmentation file.
        label_of_interest (int): The integer value of the label to analyze.
    """
    # Load the NIfTI file
    nifti_image = nib.load(segmentation_filepath)
    segmentation_data = nifti_image.get_fdata()
    
    # --- 1. Create a Binary Mask ---
    # Isolate the specific label you're interested in
    binary_mask = (segmentation_data == label_of_interest).astype(np.uint8)

    # --- 2. Calculate Thickness using SciPy ---
    # The distance_transform_edt function calculates the exact Euclidean distance
    # for each point in the mask to the nearest background point (zero).
    # We use the voxel spacing from the header for measurements in mm.
    voxel_spacing = nifti_image.header.get_zooms()
    print(f"Voxel Spacing: {voxel_spacing} mm")
    distance_array_mm = ndimage.distance_transform_edt(binary_mask)

    print(distance_array_mm)
    # --- 3. Analyze the Results ---
    shell_distances = distance_array_mm[binary_mask == 1]
    
    if len(shell_distances) == 0:
        print(f"No voxels found for label {label_of_interest}. Cannot calculate thickness.")
        return

    average_radius = np.mean(shell_distances)
    average_thickness = average_radius * 2
    max_radius = np.max(shell_distances)
    max_thickness = max_radius * 2

    print(f"--- Results for Label {label_of_interest} ---")
    print(f"Average Thickness: {average_thickness:.2f} mm")
    print(f"Maximum Thickness: {max_thickness:.2f} mm")

    # --- 4. Visualize the Process ---
    # Find a slice with the label to visualize
    slice_index = int(np.where(binary_mask == 1)[0].mean())

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(binary_mask[slice_index, :, :], cmap='gray')
    plt.title(f'Binary Mask (Slice {slice_index})')
    plt.colorbar()

    plt.subplot(1, 2, 2)
    img = plt.imshow(distance_array_mm[slice_index, :, :], cmap='viridis')
    plt.title('Distance Transform Map')
    plt.colorbar(img, label='Distance to boundary (mm)')
    
    plt.suptitle(f'Thickness Visualization for Label {label_of_interest}')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


# --- How to use it with your data ---
# Replace with the actual path to your file and the correct label number
# calculate_and_visualize_thickness('path/to/your/segmentation.nii.gz', 1) # For myocardium
# calculate_and_visualize_thickness('path/to/your/segmentation.nii.gz', 2) # For endocardium

In [18]:
a = np.array(([0,1,1,1,1],
              [0,0,1,1,1],
              [0,1,1,1,1],
              [0,1,1,1,0],
              [0,1,1,0,0]))
edt, inds = ndimage.distance_transform_edt(a,sampling=[2,1], return_indices=True)
inds

array([[[0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 3, 3],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]],

       [[0, 0, 0, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 0, 0, 4, 4],
        [0, 0, 0, 4, 4],
        [0, 0, 3, 3, 4]]], dtype=int32)

In [19]:
edt

array([[0.        , 1.        , 2.        , 2.82842712, 3.60555128],
       [0.        , 0.        , 1.        , 2.        , 3.        ],
       [0.        , 1.        , 2.        , 2.23606798, 2.        ],
       [0.        , 1.        , 2.        , 1.        , 0.        ],
       [0.        , 1.        , 1.        , 0.        , 0.        ]])

In [13]:
((2-1)**2+(3-1)**2)**0.5

2.23606797749979

In [ ]:
calculate_and_visualize_thickness("hollow_sphere.nii.gz",1)

In [ ]:

import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import nibabel as nib # Use nibabel to load your NIfTI files
# --- 1. SETUP: Create an empty 3D array and define coordinates ---
image_size = [100, 100, 100]
center = [50, 50, 50]
numpy_array = np.zeros(image_size, dtype=np.uint8) # The empty canvas
z, y, x = np.ogrid[0:image_size[0], 0:image_size[1], 0:image_size[2]] # Voxel coordinates

# --- 2. DEFINE SHAPES: Define the radii for our hollow sphere ---
outer_radius = 40
inner_radius = 30

# --- 3. CREATE STENCILS: Make boolean masks for each sphere ---
# A boolean mask where True means "inside the big sphere"
outer_sphere_mask = (x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2 <= outer_radius**2
# A boolean mask where True means "inside the small sphere"
inner_sphere_mask = (x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2 <= inner_radius**2

# --- 4. SUBTRACTION: Create the hollow shell by subtracting the inner from the outer ---
# This results in a final mask that is True only for the voxels in the shell
hollow_sphere = outer_sphere_mask & ~inner_sphere_mask

# --- 5. APPLY TO IMAGE: Use the final mask to "paint" the shell onto our canvas ---
# This sets all voxels corresponding to the shell to a value of 1
numpy_array[hollow_sphere] = 1

In [ ]:
# visualize the hollow sphere in 3d
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.voxels(numpy_array, facecolors='cyan', edgecolors='k')
ax.set_title('Hollow Sphere (3D View)')
plt.show()


In [ ]:
affine = np.eye(4)

# Create the NIfTI image object from the array and the affine matrix.
nifti_image = nib.Nifti1Image(numpy_array, affine)

# --- 3. Save the Image to a File ---
output_filename = 'hollow_sphere.nii.gz'
nib.save(nifti_image, output_filename)

print(f"✅ Successfully saved the 3D image to: {output_filename}")

In [ ]:
import napari 
viewer = napari.Viewer()
img = nib.load('hollow_sphere.nii.gz').get_fdata()
viewer.add_image(img, name='Hollow Sphere')